In [160]:
# Importing libraries
import gymnasium as gym
import numpy as np
from itertools import count
from collections import namedtuple
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [161]:
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])

In [162]:
env = gym.make('CartPole-v1') # We make the Cartpole environment here

In [163]:
print("There are {} actions".format(env.action_space.n))

There are 2 actions


In [164]:
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(4, 128) # 4 parameters in the observation space
        self.actor = nn.Linear(128, 2) # 2 possible actions
        self.critic = nn.Linear(128, 1) # Single action value

        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        x = F.relu(self.fc1(x))
        action_prob = F.softmax(self.actor(x), dim=-1)
        state_values = self.critic(x)
        return action_prob, state_values

In [165]:
def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)
    m = Categorical(probs)
    action = m.sample()
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
    return action.item()
# Decide whether to move block left or right based on model output probabilities

In [166]:
def finish_episode():
    # Calculating losses and performing backprop
    R = 0
    saved_actions = model.saved_actions
    policy_losses = []
    value_losses = []
    returns = []

    for r in model.rewards[::-1]:
        R = r + 0.99 * R # Gamma is 0.99
        returns.insert(0, R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()

        policy_losses.append(-log_prob * advantage)
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))

    optimizer.zero_grad()
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

    loss.backward()
    optimizer.step()

    del model.rewards[:]
    del model.saved_actions[:]

In [167]:
model = ActorCritic()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()

In [168]:
def train():
    running_reward = 10
    for i_episode in count():
        state, info = env.reset()
        ep_reward = 0
        for t in range(1, 10000):
            action = select_action(state)
            state, reward, terminated, truncated, info = env.step(action)
            model.rewards.append(reward)
            ep_reward += reward
            if terminated or truncated:
                break

        running_reward = 0.05 * ep_reward + (1-0.05) * running_reward
        finish_episode()
        if i_episode % 10 == 0:
            print(f"Episode {i_episode} Reward: {ep_reward:.2f} Average reward: {running_reward:.2f}")

        if running_reward > env.spec.reward_threshold:
            print(f"Solved, running reward is now {running_reward} and the last episode runs to {t} timesteps")
            break

In [169]:
train()

Episode 0 Reward: 22.00 Average reward: 10.60
Episode 10 Reward: 18.00 Average reward: 13.06
Episode 20 Reward: 9.00 Average reward: 12.06
Episode 30 Reward: 14.00 Average reward: 11.98
Episode 40 Reward: 21.00 Average reward: 12.51
Episode 50 Reward: 213.00 Average reward: 31.72
Episode 60 Reward: 62.00 Average reward: 56.30
Episode 70 Reward: 30.00 Average reward: 74.57
Episode 80 Reward: 135.00 Average reward: 95.57
Episode 90 Reward: 151.00 Average reward: 109.42
Episode 100 Reward: 13.00 Average reward: 127.43
Episode 110 Reward: 144.00 Average reward: 140.25
Episode 120 Reward: 56.00 Average reward: 145.25
Episode 130 Reward: 108.00 Average reward: 119.62
Episode 140 Reward: 500.00 Average reward: 149.30
Episode 150 Reward: 134.00 Average reward: 197.46
Episode 160 Reward: 500.00 Average reward: 283.65
Episode 170 Reward: 49.00 Average reward: 269.16
Episode 180 Reward: 98.00 Average reward: 286.02
Episode 190 Reward: 116.00 Average reward: 278.67
Episode 200 Reward: 339.00 Avera